In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
import statsmodels.api as sm

loc = 'Dataset//'

In [6]:
to_use = pd.read_csv("to_use.csv")

fixture_list = pd.read_csv("fixture_list.csv", index_col=False)

team_data = pd.read_csv("team_data.csv")

to_use = to_use.drop("Unnamed: 0", axis = 1)

fixture_list = fixture_list.drop("Unnamed: 0", axis = 1)

team_data.set_index('Team', inplace = True)

last_season_table= pd.DataFrame(index=team_data.index)


to_use

,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,0.113636,0.481481,1.000000,0.071429,0.008117,0.481481,1
1,0.340909,0.703704,0.750000,0.500000,0.170455,0.527778,0
2,0.090909,0.666667,0.291667,0.321429,0.029221,0.194444,-1
3,0.227273,1.000000,0.041667,0.785714,0.178571,0.041667,-1
4,0.454545,0.148148,0.000000,1.000000,0.454545,0.000000,-1
...,...,...,...,...,...,...,...
375,0.000000,0.629630,0.250000,0.964286,0.000000,0.157407,-1
376,0.227273,1.000000,0.916667,0.785714,0.178571,0.916667,1
377,0.159091,0.629630,0.208333,0.535714,0.085227,0.131173,-1
378,0.454545,0.000000,0.541667,0.285714,0.129870,0.000000,-1


In [9]:
X_data = to_use.drop('Result', axis =1)
Y_data = to_use['Result']

model = MultinomialNB()
model.fit(X_data, Y_data)
model.score(X_data, Y_data)

0.5631578947368421

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

model1 = MultinomialNB()
model1.fit(X_train, Y_train)

MultinomialNB()

In [12]:
predicted = model1.predict(X_test)
predicted


array([ 1, -1, -1, -1, -1,  1, -1,  1, -1,  1,  1,  1, -1, -1,  1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1, -1,  1, -1,  1,
        1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1])

In [13]:
acc_score = accuracy_score(Y_test, predicted)
print("Accuracy score: " + str(acc_score))
scores = cross_val_score(model1, X_train, Y_train, cv=10)
print("Cross val scores")
print(scores)

Accuracy score: 0.5921052631578947
Cross val scores
[0.61290323 0.51612903 0.51612903 0.5483871  0.6        0.63333333
 0.46666667 0.53333333 0.56666667 0.56666667]


In [14]:
score_train = model1.score(X_train, Y_train)
score_test = model1.score(X_test, Y_test)

print("Training set accuracy: ", '%.3f'%(score_train))
print("Test set accuracy: ", '%.3f'%(score_test))

Training set accuracy:  0.556
Test set accuracy:  0.592


In [15]:
predicted = model1.predict(X_data)
predicted
predictedDF = pd.DataFrame(predicted, columns=['Predicted'])
predictedDF.head()

,Predicted
0,1
1,-1
2,-1
3,-1
4,-1


In [16]:
to_use = to_use.merge(predictedDF, left_index=True, right_index=True)
fixture_list = fixture_list.merge(predictedDF, left_index=True, right_index=True)
fixture_list

,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result,Predicted
0,Crystal Palace,Arsenal,0.113636,0.481481,1.000000,0.071429,0.008117,0.481481,1,1
1,Fulham,Liverpool,0.340909,0.703704,0.750000,0.500000,0.170455,0.527778,0,-1
2,Bournemouth,Aston Villa,0.090909,0.666667,0.291667,0.321429,0.029221,0.194444,-1,-1
3,Leeds,Wolves,0.227273,1.000000,0.041667,0.785714,0.178571,0.041667,-1,-1
4,Newcastle,Nott'm Forest,0.454545,0.148148,0.000000,1.000000,0.454545,0.000000,-1,-1
...,...,...,...,...,...,...,...,...,...,...
375,Everton,Bournemouth,0.000000,0.629630,0.250000,0.964286,0.000000,0.157407,-1,-1
376,Leeds,Tottenham,0.227273,1.000000,0.916667,0.785714,0.178571,0.916667,1,1
377,Leicester,West Ham,0.159091,0.629630,0.208333,0.535714,0.085227,0.131173,-1,-1
378,Man United,Fulham,0.454545,0.000000,0.541667,0.285714,0.129870,0.000000,-1,-1


In [17]:
last_season_table['Actual Points'] = 0 
last_season_table['Predicted Points'] = 0 

def make_actual_table(x, ht, at, which):
    if x == -1:
        last_season_table.loc[ht, which] += 3
    elif x == 0:
        last_season_table.loc[ht, which] += 1
        last_season_table.loc[at, which] += 1
    elif x == 1:
        last_season_table.loc[at, which] += 3

fixture_list.apply(lambda x: make_actual_table(x.Result, x.HomeTeam, x.AwayTeam, 'Actual Points'), axis=1)
fixture_list.apply(lambda x: make_actual_table(x.Predicted, x.HomeTeam, x.AwayTeam, 'Predicted Points'), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

In [18]:
actual_index = last_season_table.sort_values(by='Actual Points', ascending=0).index
predicted_index = last_season_table.sort_values(by='Predicted Points', ascending=0).index

last_season_table.sort_values(by='Actual Points', ascending=0)

,Actual Points,Predicted Points
Team,,
Man City,89,93
Arsenal,84,93
Man United,75,60
Newcastle,71,87
Liverpool,67,69
Brighton,62,72
Aston Villa,61,57
Tottenham,60,63
Brentford,59,60


In [19]:
def acc(act, prd):
    count = 0
    for i, val in enumerate(act):
        if str(act[i]) == str(prd[i]):
            count += 1
    print("Accuracy of Predicted Table to Accurate Table " + str((count/20)*100) + "%")

acc(actual_index,predicted_index)

# 18/19 Season predicted table
last_season_table.sort_values(by='Predicted Points', ascending =0)

Accuracy of Predicted Table to Accurate Table 10.0%


,Actual Points,Predicted Points
Team,,
Arsenal,84,93
Man City,89,93
Newcastle,71,87
Brighton,62,72
Liverpool,67,69
Tottenham,60,63
Man United,75,60
Brentford,59,60
Fulham,52,57
